In [1]:
import json
import os
import numpy as np

In [2]:
def import_metrics_json(models_path, run_num):
    """
    Looks in {models_path}/{run_num}/metrics.json and returns the contents as a
    Python dictionary. Returns None if the path does not exist.
    """
    path = os.path.join(models_path, str(run_num), "metrics.json")
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        return json.load(f)

In [3]:
def get_best_metric(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value to use for
    comparison. The best metric value is determined by `metric_compare_func`, which
    must take in two arguments, and return whether or not the _first_ one is better.
    Returns the number of the run, the value associated with that run, and a dict of
    all the values used for comparison.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_val, all_vals = None, None, {}
    for run_num in metrics.keys():
        try:
            val = reduce_func(metrics[run_num][metric_name]["values"])
            all_vals[run_num] = val
            if best_val is None or compare_func(val, best_val):
                best_val, best_run = val, run_num
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_val, all_vals

In [4]:
def get_best_metric_at_best_epoch(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value FOR EACH
    SUBARRAY/VALUE in the value array to use for comparison. The best metric value
    is determined by `metric_compare_func`, which must take in two arguments, and
    return whether or not the _first_ one is better.
    Returns the number of the run, the (one-indexed) number of the epch, the value
    associated with that run and epoch, and a dict of all the values used for
    comparison (mapping pair of run number and epoch number to value).
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_epoch, best_val, all_vals = None, None, None, {}
    for run_num in metrics.keys():
        try:
            # Find the best epoch within that run
            best_epoch_in_run, best_val_in_run = None, None
            for i, subarr in enumerate(metrics[run_num][metric_name]["values"]):
                val = reduce_func(subarr)
                if best_val_in_run is None or compare_func(val, best_val_in_run):
                    best_epoch_in_run, best_val_in_run = i + 1, val
            all_vals[(run_num, best_epoch_in_run)] = best_val_in_run
            
            # If the best value in the best epoch of the run is best so far, update
            if best_val is None or compare_func(best_val_in_run, best_val):
                best_run, best_epoch, best_val = run_num, best_epoch_in_run, best_val_in_run
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_epoch, best_val, all_vals

In [9]:
models_path = "/users/amtseng/att_priors/models/trained_models/profile_models/SPI1_prior/"
best_run, best_epoch, best_val, all_vals = get_best_metric_at_best_epoch(
    models_path,
    "val_prof_corr_losses",
    lambda values: np.mean(values),
    lambda x, y: x < y
)
print("Best run: %s" % best_run)
print("Best epoch in run: %d" % best_epoch)
print("Associated value: %s" % best_val)
for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
    print(key, all_vals[key])

Best run: 7
Best epoch in run: 1
Associated value: 89.94859203334525
('1', 5) 92.87491922347809
('2', 2) 91.81298261861669
('3', 3) 90.98503513520565
('4', 4) 91.65070682283887
('5', 7) 92.79284542587723
('6', 1) 90.31572375108041
('7', 1) 89.94859203334525
('8', 1) 90.88748335300532
('9', 3) 94.3520067339937
('10', 1) 90.28025521627686
('11', 10) 93.6835513662959
('12', 2) 91.68056812552709
('13', 1) 90.3932429886274
('14', 1) 90.75675676716642
('15', 6) 92.59214978571481


In [11]:
for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
    print(key[0])
    metrics = import_metrics_json(models_path, key[0])
    print(np.mean(metrics["val_prof_corr_losses"]["values"], axis=1))
    print(np.mean(metrics["val_pos_att_losses"]["values"], axis=1))

1
[94.37510603 93.54147481 93.86899592 92.90773183 92.87491922 93.15681116
 93.34095781 93.13254945 94.30564022 93.53211262]
[0.23614891 0.07265516 0.05968027 0.04774254 0.04080145 0.03647423
 0.03300492 0.03081511 0.02749416 0.027668  ]
2
[ 92.35735941  91.81298262  93.71755831  93.8992644   94.86775387
  96.28630551  98.04736373 102.45802585 106.65216831 125.23091452]
[0.14959264 0.06180098 0.04442908 0.02954582 0.03102257 0.02951256
 0.02865808 0.02427959 0.02407855 0.24149895]
3
[91.73066505 91.22965849 90.98503514 91.08862228 91.74752596 91.78483883
 92.40955055 92.27776492 94.53128618 93.94874115]
[0.16432875 0.06235689 0.04442935 0.03651058 0.0315486  0.03033786
 0.02869359 0.02991026 0.02721583 0.02920703]
4
[92.18479015 92.03698236 91.6579977  91.65070682 91.80126954 92.78554964
 91.71918688 92.11625637 92.87797818 93.136613  ]
[0.1871363  0.06845363 0.04990551 0.0386693  0.03243457 0.0281125
 0.02836284 0.0293305  0.02916564 0.02901448]
5
[93.57477461 93.27071851 92.95506118 